In [1]:
# %load_ext autoreload
# %autoreload 2
# %matplotlib inline
# import matplotlib.pyplot as plt
import sys; sys.path.append('../')
from misc import h5file

import numpy as np
from numpy.random import default_rng
import scipy.io as sio

import torch, sympytorch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import grad
from siren_pytorch import SirenNet

import pysindy as ps

from sympy import symbols, simplify, lambdify
from mathparser import math_eval
from varname import nameof

import sys; sys.path.append('../optimizers/')
from Adan import Adan
from FNO_Adam import Adam

MAIN_SEED = 1234

In [2]:
X_pre, best_subsets, y_pre = h5file(file_path="../PMS_data/burgers_pms.h5", 
                                    mode='r', return_dict=False)

['X_pre', 'best_subsets', 'y_pre']


In [3]:
un = sio.loadmat("../RDAE_data/l21rdae_noisy40_burgers_0.5.mat")["rdae_denoised_un"].real

In [4]:
data = sio.loadmat('../Datasets/burgers.mat')
x = (data['x'][0]).real
t = (data['t'][:,0]).real
dt = t[1]-t[0]; dx = x[2]-x[1]
X, T = np.meshgrid(x, t)
XT = np.asarray([X, T]).T
del data

In [5]:
import yaml
with open("../PMS_data/burgers_pms_feature_names.yaml", 'r') as f:
    config = yaml.load(f, yaml.Loader)
f.close()
encoded_feature_names = config["encoded_feature_names"]
encoded_pde_names = config["encoded_pde_names"]

In [6]:
encoded_pde_names

['u*u_1',
 'u_11+u*u_1',
 'u_11+u*u_1+u*u_11',
 'u_11+u*u_1+u*u_11+u*u*u_11',
 'u+u_11+u*u_1+u*u_11+u*u*u_11',
 'u+u_11+u*u_1+u*u*u_1+u*u_11+u*u*u_11',
 'u+u*u+u_11+u*u_1+u*u*u_1+u*u_11+u*u*u_11',
 'u+u*u+u_1+u_11+u*u_1+u*u*u_1+u*u_11+u*u*u_11']

In [7]:
def log_like_value(prediction, ground):                                                                                                               
    nobs = float(ground.shape[0])
    nobs2 = nobs / 2.0
    ssr = np.sum(np.abs(ground - prediction)**2)
    llf = -nobs2 * np.log(2 * np.pi) - nobs2 * np.log(ssr / nobs) - nobs2
    return llf

def BIC_AIC(prediction, ground, nparams, reg_func = lambda x: x):
    nparams = reg_func(nparams)
    llf = log_like_value(prediction, ground)
    return -2*llf + np.log(ground.shape[0])*nparams, -2*llf + 2*nparams

In [8]:
class PhysicalConstraintCalculator(nn.Module):
    def __init__(self, symbolic_module, basic_vars, init_coefficients=None, learnable_coefficients=False):
        super(PhysicalConstraintCalculator, self).__init__()
        self.symbolic_module = symbolic_module
        self.basic_vars = basic_vars
        
        self.coefficients = init_coefficients
        self.learnable_coefficients = learnable_coefficients

        if self.coefficients is None:
            self.coefficients = torch.ones(len(symbolic_module.sympy())).float()
        else:
            self.coefficients = torch.tensor(data=self.coefficients).float()
        self.coefficients = nn.Parameter(self.coefficients).requires_grad_(self.learnable_coefficients)
        
        # printing
        if self.learnable_coefficients: print("Learnable coefficients:", self.coefficients)
        else: print("NOT learnable coefficients:", self.coefficients)
        print(symbolic_module.sympy())
        print("Basic variables:", self.basic_vars)

    def set_learnable_coefficients(self, learn):
        self.coefficients.requires_grad_(learn)
    
    def forward(self, input_dict):
        return self.symbolic_module(**input_dict)

In [9]:
class Sine(nn.Module):
    def __init__(self, ):
        super(Sine, self).__init__()
    def forward(self, x):
        return torch.sin(x)

class TorchMLP(nn.Module):
    def __init__(self, dimensions, bias=True, activation_function=nn.Tanh(), bn=None, dropout=None):
        super(TorchMLP, self).__init__()
        # setup ModuleList
        self.model  = nn.ModuleList()
        for i in range(len(dimensions)-1):
            self.model.append(nn.Linear(dimensions[i], dimensions[i+1], bias=bias))
            if bn is not None and i!=len(dimensions)-2:
                self.model.append(bn(dimensions[i+1]))
                if dropout is not None:
                    self.model.append(dropout)
            if i==len(dimensions)-2: break
            self.model.append(activation_function)
        # weight init
        self.model.apply(self.xavier_init)

    def xavier_init(self, m):
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.0)

    def forward(self, x):
        for i, l in enumerate(self.model): 
            x = l(x)
        return x

In [10]:
# Implement DeepONet(nn.Module)

class PINN(nn.Module):
    def __init__(self, solver, physics_calculator, lb, ub, 
                 domain_dimension=None, weak_pde_lib=None, effective_indices=None):
        super(PINN, self).__init__()
        self.solver = solver
        self.physics_calculator = physics_calculator
        self.lb = lb
        self.ub = ub
        # Only to use weak_loss
        # spatial x temporal
        self.domain_dimension = domain_dimension
        self.weak_pde_lib = weak_pde_lib
        self.effective_indices = effective_indices
        self.weak_coeff_buffer = None
        
    def forward(self, x, t):
        return self.solver(self.input_normalize(torch.cat([x, t],  dim=-1)))

    def calculate_physics(self, x, t):
        u = self.forward(x, t)
        u_t = self.gradients(u, t)[0]
        u_1 = self.gradients(u, x)[0]
        u_11 = self.gradients(u_1, x)[0]
        physics = self.physics_calculator({nameof(u):u, 
                                           nameof(u_1):u_1, 
                                           nameof(u_11):u_11})
        
        return u, u_t, physics
    
    def loss(self, x, t, y_input):
        u, u_t, physics = self.calculate_physics(x, t)
        coeff = self.physics_calculator.coefficients
        physics = (physics*coeff).sum(axis=-1)
        mse = F.mse_loss(u, y_input, reduction='mean')
        l_eq = F.mse_loss(u_t, physics, reduction='mean')
        return torch.add(mse, l_eq)
    
    def weak_loss(self, x, t, y_input):
        u, u_t, physics = self.calculate_physics(x, t)
        coeff = torch.tensor(self.weak_coefficients(u)).float()
        physics = (physics*coeff).sum(axis=-1)
        mse = F.mse_loss(u, y_input, reduction='mean')
        l_eq = F.mse_loss(u_t, physics, reduction='mean')
        return torch.add(mse, l_eq)
    
    def weak_form(self, u):
        pred = u.reshape(self.domain_dimension[1], 
                         self.domain_dimension[0]).T.detach().numpy()
        pred = np.expand_dims(pred,-1)
        X_weak = self.weak_pde_lib.fit_transform(pred)
        y_weak = self.weak_pde_lib.convert_u_dot_integral(pred)
        return X_weak, y_weak
    
    def weak_coefficients(self, u):
        np.random.seed(0)
        X_weak, y_weak = self.weak_form(u)
        X_weak = X_weak[:, self.effective_indices]
        self.weak_coeff_buffer = np.linalg.lstsq(X_weak, y_weak, rcond=None)[0].flatten()
        return self.weak_coeff_buffer
    
    def gradients(self, func, x):
        return grad(func, x, create_graph=True, retain_graph=True, 
                    grad_outputs=torch.ones(func.shape))

    def input_normalize(self, inp):
        return -1.0+2.0*(inp-self.lb)/(self.ub-self.lb)

In [11]:
rng = default_rng(seed=0)
# sampled_indices_x = rng.choice(len(x), size=int(len(x)//(2)), replace=False)
# sampled_indices_t = rng.choice(len(t), size=int(len(t)//(2)), replace=False)
sampled_indices_x = np.array([i for i in range(len(x)) if i%2==0])
sampled_indices_t = np.array([i for i in range(len(t)) if i%2==0]) 
domain_dimension = len(sampled_indices_x), len(sampled_indices_t)

In [12]:
np.random.seed(MAIN_SEED);
torch.manual_seed(MAIN_SEED);

In [13]:
XX = X[sampled_indices_t, :][:, sampled_indices_x]
TT = T[sampled_indices_t, :][:, sampled_indices_x]
XXTT = XT[sampled_indices_x, :, :][:, sampled_indices_t, :]

In [14]:
K = 3000; diff_order = 2
weak_pde_lib = ps.WeakPDELibrary(library_functions=[lambda x:x, lambda x: x*x], 
                                 function_names=[lambda x:x, lambda x: x+x], 
                                 derivative_order=diff_order, p=diff_order, 
                                 spatiotemporal_grid=XXTT, 
                                 include_bias=False, is_uniform=True, K=K # new random K points in every calls to the ps.WeakPDELibrary
                                )

In [15]:
X_train = np.hstack((XX.flatten()[:,None], TT.flatten()[:,None]))
y_train = un.T[sampled_indices_t, :][:, sampled_indices_x].flatten()[:,None]
lb = torch.tensor(X_train.min(axis=0)).float().requires_grad_(False)
ub = torch.tensor(X_train.max(axis=0)).float().requires_grad_(False)

In [16]:
del XX, TT, XXTT

In [17]:
# Converting to tensors
X_train = torch.tensor(X_train).float().requires_grad_(True)
y_train = torch.tensor(y_train).float().requires_grad_(False)
X_train.shape, y_train.shape

(torch.Size([6528, 2]), torch.Size([6528, 1]))

In [18]:
com = 2; com = max(com, 1)
effective_indices = np.where(best_subsets[com-1]>0)[0].tolist()
init_coefficients = np.linalg.lstsq(X_pre[:, effective_indices], 
                                    y_pre, rcond=None)[0].flatten()
mod, basic_vars = math_eval(encoded_pde_names[com-1], 
                            return_torch=True, split_by_addition=True)
init_coefficients, mod

(array([ 0.09517879, -1.0106909 ], dtype=float32),
 SymPyModule(expressions=(u_11, u*u_1)))

In [19]:
# bias init at 0.01 | SIREN
# activation_function = nn.Tanh()
activation_function = Sine()
solver = TorchMLP([2,50,50,50,50,1], bn=None, activation_function=activation_function)
# solver = SirenNet(dim_in=2, dim_hidden=50, dim_out=1, num_layers = 4, 
#                   w0_initial = 30.)
physics_calculator = PhysicalConstraintCalculator(symbolic_module=mod, 
                                                  basic_vars=basic_vars, 
                                                  init_coefficients=init_coefficients, 
                                                  learnable_coefficients=True)

Learnable coefficients: Parameter containing:
tensor([ 0.0952, -1.0107], requires_grad=True)
[u_11, u*u_1]
Basic variables: ['u', 'u_1', 'u_11']


In [20]:
pinn = PINN(solver, physics_calculator, 
            lb, ub, domain_dimension, 
            weak_pde_lib, effective_indices)

In [21]:
# pinn.loss(X_train[:, 0:1], X_train[:, 1:2], y_train)
# pinn.weak_loss(X_train[:, 0:1], X_train[:, 1:2], y_train)

In [22]:
pinn.physics_calculator.set_learnable_coefficients(False)
lbfgs = torch.optim.LBFGS(pinn.parameters(), lr=0.1, 
                          max_iter=500, max_eval=500, history_size=300, 
                          line_search_fn='strong_wolfe')
epochs = 100
pinn.train()

for i in range(epochs):
    def closure():
        if torch.is_grad_enabled(): 
            lbfgs.zero_grad()
        l = pinn.loss(X_train[:, 0:1], X_train[:, 1:2], y_train)
        if l.requires_grad: 
            l.backward()
        return l

    lbfgs.step(closure)

    # calculate the loss again for monitoring
    if (i%50)==0 or i==epochs-1:
        l = closure()
        print("Epoch {}: ".format(i), l.item())

Epoch 0:  0.0025768722407519817
Epoch 50:  0.0023282042238861322
Epoch 99:  0.0023282042238861322


In [23]:
pinn.eval()
pred = pinn(X_train[:, 0:1], X_train[:, 1:2]).detach().numpy()
BIC_AIC(pred, y_train.detach().numpy(), com)

(-21077.626061478906, -21091.193773272193)

In [24]:
# torch.save(pinn.state_dict(), "../PMS_weights/pinn_mlp_sine.pth")

#### weak

In [25]:
### using lbfgs ###
print("using lbfgs...")
epochs = 100
pinn.train()
pinn.physics_calculator.set_learnable_coefficients(False)
lbfgs2 = torch.optim.LBFGS(pinn.parameters(), lr=0.1, 
                          max_iter=500, max_eval=500, history_size=300, 
                          line_search_fn='strong_wolfe')

for i in range(epochs):
    def closure2():
        if torch.is_grad_enabled(): 
            lbfgs2.zero_grad()
        l = pinn.weak_loss(X_train[:, 0:1], X_train[:, 1:2], y_train)
        if l.requires_grad: 
            l.backward()
        return l

    lbfgs2.step(closure2)

    # calculate the loss again for monitoring
    if (i%50)==0 or i==epochs-1:
        l = closure2()
        print("Epoch {}: ".format(i), l.item())

pinn.eval()
pred = pinn(X_train[:, 0:1], X_train[:, 1:2]).detach().numpy()
print(BIC_AIC(pred, y_train.detach().numpy(), com))
print(pinn.physics_calculator.coefficients.detach().numpy())
print(pinn.weak_coeff_buffer)

### using non-lbfgs ###
print("using non-lbfgs...")
epochs = 1000
pinn.train()
pinn.physics_calculator.set_learnable_coefficients(False)
# optimizer = Adam(pinn.parameters(), lr=1e-5, weight_decay=1e-4)
optimizer = Adan(pinn.parameters(), lr=1e-5, weight_decay=1e-2)

for i in range(epochs):
    optimizer.zero_grad()
    l = pinn.weak_loss(X_train[:, 0:1], X_train[:, 1:2], y_train)
    l.backward()
    optimizer.step()
    if (i%50)==0 or i==epochs-1:
        l = pinn.weak_loss(X_train[:, 0:1], X_train[:, 1:2], y_train)
        print("Epoch {}: ".format(i), l.item())

using lbfgs...
Epoch 0:  0.0023282102774828672
Epoch 50:  0.0023282060865312815
Epoch 99:  0.0023282060865312815
(-21077.95679626977, -21091.524508063056)
[ 0.09517879 -1.0106909 ]
[ 0.09809884 -1.00043748]
using non-lbfgs...
Epoch 0:  0.0023289392702281475
Epoch 50:  0.0023281299509108067
Epoch 100:  0.0023280950263142586
Epoch 150:  0.0023280729074031115
Epoch 200:  0.0023280379828065634
Epoch 250:  0.002327998634427786
Epoch 300:  0.0023279767483472824
Epoch 350:  0.002327963709831238
Epoch 400:  0.002327953465282917
Epoch 450:  0.002327939961105585
Epoch 500:  0.002327929250895977
Epoch 550:  0.00232792180031538
Epoch 600:  0.002327914349734783
Epoch 650:  0.002327903173863888
Epoch 700:  0.002327897120267153
Epoch 750:  0.002327889436855912
Epoch 800:  0.002327884081751108
Epoch 850:  0.002327875467017293
Epoch 900:  0.002327869413420558
Epoch 950:  0.0023278642911463976
Epoch 999:  0.0023278568405658007


#### learn (pinn.physics_calculator.set_learnable_coefficients(True))

In [26]:
### using non-lbfgs ###
# epochs = 1000
# pinn.train()
# pinn.physics_calculator.set_learnable_coefficients(True)
# # optimizer = Adam(pinn.parameters(), lr=1e-5, weight_decay=1e-4)
# optimizer = Adan(pinn.parameters(), lr=1e-5, weight_decay=1e-2)

# for i in range(epochs):
#     optimizer.zero_grad()
#     l = pinn.loss(X_train[:, 0:1], X_train[:, 1:2], y_train)
#     l.backward()
#     optimizer.step()
#     if (i%50)==0 or i==epochs-1:
#         l = pinn.loss(X_train[:, 0:1], X_train[:, 1:2], y_train)
#         print("Epoch {}: ".format(i), l.item())

### using lbfgs ###
# epochs = 100
# pinn.train()
# pinn.physics_calculator.set_learnable_coefficients(True)
# lbfgs2 = torch.optim.LBFGS(pinn.parameters(), lr=0.1, 
#                           max_iter=500, max_eval=500, history_size=300, 
#                           line_search_fn='strong_wolfe')

# for i in range(epochs):
#     def closure2():
#         if torch.is_grad_enabled(): 
#             lbfgs2.zero_grad()
#         l = pinn.loss(X_train[:, 0:1], X_train[:, 1:2], y_train)
#         if l.requires_grad: 
#             l.backward()
#         return l

#     lbfgs2.step(closure2)

#     # calculate the loss again for monitoring
#     if (i%50)==0 or i==epochs-1:
#         l = closure2()
#         print("Epoch {}: ".format(i), l.item())

#### eval

In [27]:
pinn.eval()
pred = pinn(X_train[:, 0:1], X_train[:, 1:2]).detach().numpy()
print(BIC_AIC(pred, y_train.detach().numpy(), com))
print(pinn.physics_calculator.coefficients.detach().numpy())
print(pinn.weak_coeff_buffer)

(-21077.070613192496, -21090.638324985783)
[ 0.09517879 -1.0106909 ]
[ 0.09902192 -1.00018011]


In [32]:
# torch.save(pinn.state_dict(), "../PMS_weights/pinn_mlp_sine_weak.pth")

In [33]:
# def percent_coeff_error(pred):
#     ground = np.array([0.1, -1])
#     errs = 100*np.abs(np.array(pred)-ground)/np.abs(ground)
#     return errs.mean(), errs.std()
# print(percent_coeff_error([ 0.09517879, -1.0106909 ]))
# print(percent_coeff_error([ 0.09809884, -1.00043748]))
# print(percent_coeff_error([ 0.09768956, -1.00105876]))
# print(percent_coeff_error([ 0.0990209, -1.00016034]))

#### Notes
    - 3 main files are required.

In [34]:
# torch.save(pinn.state_dict(), "../PMS_weights/pinn_mlp_sine_weak.pth")
# (-21077.95679626977, -21091.524508063056)
# [[ 0.09809884]
#  [-1.00043748]]

In [35]:
# nn.Tanh(): (-20637.284432281755, -20650.81253882045)
# Sine(): (-20681.104197432018, -20694.632303970713)
# SirenNet(dim_in = 2, dim_hidden = 50, dim_out = 1, num_layers = 4, w0_initial = 30.): (-21585.25976244083, -21598.787868979525)